## Expanding Window

In this notebook, we will use an expanding window method vs a sliding window method. Modifications to be made are as follows:

1. Training set needs to become daily+. Currently it counts each patient's daily diagnostics starting from day 0, day 1,...day n. It will have to be expanded to be from day0_new= day0, day1_new= min(day0, day1), day2_new= min(day0, day1, day2), etc.
2. Testing set also needs to be modified the same way, specifically for hour 24 and on. So instead of taking the minimums from the last 24 hours, it will take the minimums from the last n hours.

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
hadm_id = np.load('C:/Users/faith/OneDrive/Documents/FaithZhang/SURF/RealTimeDetection/Data/all_ids.npy')
# randomly spliated 70-30
np.random.seed(123) # set seed for reproducibility 
train_ids = np.random.choice( hadm_id, round(len(hadm_id)*0.7), replace = False )
test_ids = np.array(list(set(hadm_id) - set(train_ids)))

print("all patient ids", hadm_id, 
      "; the total number of patients being 4120, \n which is 147 short from the 4267 supposed patients stated in the article :(")

all patient ids [20001687 20005241 20006999 ... 29994991 29997844 29998115] ; the total number of patients being 4120, 
 which is 147 short from the 4267 supposed patients stated in the article :(


In [4]:
print("length of training data:", len(train_ids), '\n', "length of testing data:", len(test_ids))

length of training data: 2884 
 length of testing data: 1236


For some reason there are 491 hadm_ids?

In [44]:
# import os

# def count_files_in_folder(folder_path):
#     folder_path = os.path.join(folder_path, 'C:/Users/faith/OneDrive/Documents/FaithZhang/SURF/RealTimeDetection/Data/byID_daily')
#     num_files = len([name for name in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, name))])
#     return num_files

# folder_path = 'C:/Users/faith/OneDrive/Documents/FaithZhang/SURF/RealTimeDetection/Data/byID_daily'
# num_files = count_files_in_folder(folder_path)
# print(f'Number of files in folder "{folder_path}" is: {num_files}')


Number of files in folder "C:/Users/faith/OneDrive/Documents/FaithZhang/SURF/RealTimeDetection/Data/byID_daily" is: 491


### Revelations

Data -> byID: all 2000+ hadm_ids containing each patient's hourly data. 

Data -> byID_daily: all 2000+ hadm_ids DAILY summaries of their data (basically all of the daily minimums)

In [13]:
# prep test code for actual code below
temp = pd.read_pickle('C:/Users/faith/OneDrive/Documents/FaithZhang/SURF/RealTimeDetection/Data/byID/20955149.pkl')
temp
#this is the daily patient aki result i guess??

,hadm_id,timedelta,arterial_bp_diastolic,arterial_bp_mean,arterial_bp_systolic,cvp,heart_rate,spo2,pap_diastolic,pap_mean,...,mild_liver_disease,diabetes_without_cc,diabetes_with_cc,paraplegia,renal_disease,malignant_cancer,severe_liver_disease,metastatic_solid_tumor,aids,AKI_any
0,20955149.0,0 days 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20955149.0,0 days 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20955149.0,0 days 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20955149.0,0 days 04:00:00,62.470588,77.823529,101.529412,NaN,82.000000,100.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20955149.0,0 days 05:00:00,68.219563,86.328310,115.911052,NaN,68.954492,100.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,20955149.0,0 days 06:00:00,63.236763,81.770156,111.161252,NaN,68.709988,100.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,20955149.0,0 days 07:00:00,66.000000,85.000000,114.000000,NaN,70.000000,100.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,20955149.0,0 days 08:00:00,57.440000,72.000000,93.560000,NaN,69.320000,100.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,20955149.0,0 days 09:00:00,69.000000,95.000000,138.000000,NaN,78.000000,100.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,20955149.0,0 days 10:00:00,63.000000,84.000000,122.000000,NaN,73.000000,100.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# prep test code for actual code below
temp = pd.read_pickle('C:/Users/faith/OneDrive/Documents/FaithZhang/SURF/RealTimeDetection/Data/byID_daily/20955149.pkl')
temp

#this is the daily patient aki result i guess??

,hadm_id,arterial_bp_diastolic_min,arterial_bp_mean_min,arterial_bp_systolic_min,cvp_min,heart_rate_min,spo2_min,pap_diastolic_min,pap_mean_min,pap_systolic_min,...,mild_liver_disease,diabetes_without_cc,diabetes_with_cc,paraplegia,renal_disease,malignant_cancer,severe_liver_disease,metastatic_solid_tumor,aids,AKI_any
0,20955149.0,51.0,68.0,93.0,NaN,68.709988,94.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20955149.0,62.0,82.0,113.0,NaN,78.000000,95.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## creating the TRAINING data 

This will be daily+ data (because it will consider minimums starting from hour 0, no matter what)

Considering 'daily_data' is relatively robust (less affected by inaccurate time indicators)  
We re-arrange the data, using parameters in today, predicting AKI indicator tomorrow (24 hrs ahead)

1. Training set needs to become daily+. Currently it counts each patient's daily diagnostics starting from day 0, day 1,...day n. It will have to be expanded to be from day0_new= day0, day1_new= min(day0, day1), day2_new= min(day0, day1, day2), etc.


In [16]:
# prepare training set
training_appended = []
training_dp = pd.DataFrame()

train_raw= pd.read_csv('C:/Users/faith/OneDrive/Documents/FaithZhang/SURF/RealTimeDetection/daily_maxmin/new_data/train_set_byhour.csv')
np.unique(train_raw['hadm_id'])

timeframes= pd.DataFrame()

for idx in tqdm(range(len(train_ids))):
    if row['hadm_id']== train_ids
    for times in 



#STEPS:
# 1) create an empty training dataframe that will eventually be converted into a pickle file
# 2) loop through all of the training IDS from the HOURLY data
# 3) For each PATIENT, loop throuhg each TIME FRAME (i.e. from 0-1, 0-2,... 0-n). 
# 3a) For each of these time frames, we find the minimum feature values for each column
# 3b) We store all of this information into the new dataframe, where each row represents






# for idx in tqdm(range(len(train_ids))):
#     tem_id = train_ids[idx]
#     tem_daily = pd.read_pickle( 'C:Users/faith/OneDrive/Documents/FaithZhang/SURF/RealTimeDetection/Data/byID_daily' + str(tem_id) + '.pkl')
    
#     if len(tem_daily) > 1: #for all hadm_ids that represent patients that stayed in  the icu for more than one day
#         tem_train = pd.concat(  [tem_daily.iloc[ 0:(len(tem_daily)-1)],  #FROM DAILYIDS: take info for all rows (days) except for the last day
#                      pd.DataFrame(tem_daily[ 'AKI_any' ].iloc[ 1:len(tem_daily) ].values) ] #extract the AKI_any column values from the second row (day) to the end row (day)
#                     , axis = 1,
#                     ).drop( 'AKI_any', axis = 1) #concatenate the second line to the first line such that it is displaced (AKI_any thus becomes AKI_in_24) and the last day is missing a value
        
#         training_appended.append(tem_train) #each modified dataframe stored in tem_train and added to training_appended list
        
# training = pd.concat(training_appended) #initially empty training df gets all modified dfs added to it.
# training.rename( columns = {0:'AKI_in_24'}, inplace = True)
# training.to_pickle('./train_set.pkl')

array([20001687., 20012858., 20014283., ..., 29991038., 29997844.,
       29998115.])

In [5]:
# used a faster way by list to store everything

# # prepare training set
# training = pd.DataFrame()
# for idx in tqdm(range(len(train_ids))):
#     tem_id = train_ids[idx]
#     tem_daily = pd.read_pickle( '../Data/byID_daily/' + str(tem_id) + '.pkl')
    
#     if len(tem_daily) > 1:
#         tem_train = pd.concat(  [tem_daily.iloc[ 0:(len(tem_daily)-1)],  
#                      pd.DataFrame(tem_daily[ 'AKI_any' ].iloc[ 1:len(tem_daily) ].values) ]
#                     , axis = 1,
#                     ).drop( 'AKI_any', axis = 1)
        
#         training = pd.concat( [training, tem_train], ignore_index = True )
        
# training.rename( columns = {0:'AKI_in_24'}, inplace = True)
# training.to_pickle('./train_set.pkl')

In [6]:
# load training if loss connection
training = pd.read_pickle('./train_set.pkl')

testing = pd.DataFrame( columns = np.concatenate( [ ['time'], training.columns.values]))
inform = pd.DataFrame( columns = ['hadm_id', 'if_AKI_inICU','first_AKI_detected'])

## creating the TESTING data

testing set will be the rolling window, using ( max(0, t-24), t) to predict (t+1, t+24)  
Record additional information such as  
'if the patients develop AKI during ICU'  
'when is the first time that the patient is found AKI'  
for the future report

CREATING INFORM and TESTING datasets:

In [7]:

testing_appended = []
inform_appended = []

# prepare testing
for idx in tqdm(range(len(test_ids))):
    tem_id = test_ids[idx]
    tem_patient = pd.read_pickle( '../Data/byID/' + str(tem_id) + '.pkl')
    time_stamps = tem_patient['timedelta']
    
    tem_testing = pd.DataFrame( columns = np.concatenate( [ ['time'], training.columns.values]))
    for time_id in range(len(time_stamps)):
        if time_id != (len(time_stamps) - 1):
            tem = pd.DataFrame( data = np.concatenate( (np.array([time_id, tem_id]),
                    tem_patient.loc[ tem_patient['timedelta'].isin( time_stamps[ max(0,(time_id-23)) : (time_id+1)].values ) ].drop( ['hadm_id','timedelta','Heart Rhythm','admission_category'], axis = 1 ).min(axis = 0).values[0:547],
                    tem_patient.loc[ tem_patient['timedelta'].isin( time_stamps[ max(0,(time_id-23)) : (time_id+1)].values ) ].drop( ['hadm_id','timedelta','Heart Rhythm','admission_category'], axis = 1 ).max(axis = 0).values[0:572],
                    np.array( [tem_patient.loc[ tem_patient['timedelta'].isin( time_stamps[ (time_id+1) : min(len(time_stamps), time_id+25 )].values ), 'AKI_any' ].max()])
                    )).reshape( [1,-1] ),
                    columns = testing.columns.values)
            
            testing_appended.append(tem)

    if tem_patient['AKI_any'].max() == 0:
        new_inform = pd.DataFrame( data = np.array([tem_id, tem_patient['AKI_any'].max(), float('NaN')]).reshape([1,-1]),
                    columns = ['hadm_id', 'if_AKI_inICU','first_AKI_detected'])
    else:
        new_inform = pd.DataFrame( data = np.array([tem_id, tem_patient['AKI_any'].max(), tem_patient.loc[ tem_patient['AKI_any'] == 1, 'timedelta'].index[0]]).reshape([1,-1]),
                    columns = ['hadm_id', 'if_AKI_inICU','first_AKI_detected'])
        
    inform_appended.append(new_inform)
    

#testing.to_pickle('./testing/test_set.pkl')
#inform.to_pickle('./testing/inform.pkl')

100%|██████████| 1236/1236 [27:03<00:00,  1.31s/it] 


In [8]:
testing = pd.concat(testing_appended)
inform = pd.concat(inform_appended)

In [9]:
testing.loc[testing['gender'] == 'M', 'gender'] = 0
testing.loc[testing['gender'] == 'F', 'gender'] = 1 

In [10]:
testing = testing.astype( 'float' )
testing.to_pickle('./test_set.pkl')

In [11]:
inform.to_pickle('./inform.pkl')